In [1]:
## Import

%matplotlib inline
import numpy as np
import pandas as pd
import os
import urllib.request
import datetime
import gzip
import shutil


## Read in the data

In [3]:
listing_col = ['id',
               'listing_url',
               'name',
               'price',
               'summary',
               'host_id',
               'host_name',
               'host_about',
               'host_since',
               'host_listings_count',
               'host_total_listings_count',
               'host_response_rate',
               'description',
               'neighbourhood_cleansed',
               'property_type',
               'room_type',
               'price',
               'number_of_reviews',
               'instant_bookable',
               'review_scores_rating',
               'beds',
               'bedrooms',
               'bathrooms',
               'accommodates',
               'minimum_nights',
               'maximum_nights',
               'amenities', # Betekent voorzieningen
               'cancellation_policy',
               'reviews_per_month',
               'first_review',
               'last_review',
               'reviews_per_month',
               'review_scores_rating',
               'latitude',
               'longitude',
               'availability_30',
               'availability_60',
               'availability_90',
               'availability_365'              
              ]

url = 'http://data.insideairbnb.com/the-netherlands/north-holland/amsterdam/2018-01-10/data/listings.csv.gz'
filename = 'listings_2018-01-10.csv.gz'
urllib.request.urlretrieve(url, filename)
unfilled = gzip.open(filename)
df_listings = pd.read_csv(unfilled, usecols=listing_col)
df_listings['publicatie'] = '2018-01-10'
        

our_dates= []
start = datetime.datetime.strptime("11-01-2018", "%d-%m-%Y")  # we can change this
end = datetime.datetime.strptime("31-12-2018", "%d-%m-%Y")    # we can change this
date_generated = [start + datetime.timedelta(days=x) for x in range(0, (end-start).days)]

for date in date_generated:
    our_dates.append(date.strftime("%Y-%m-%d"))

    
item = "listings" ## We can also extract reviews and calanders
    
for date in our_dates:
    url = "http://data.insideairbnb.com/the-netherlands/north-holland/amsterdam/" +date+ "/data/"+item+".csv.gz"
    filename = item +"_"+ date +".csv.gz" 
    file_python = item +"_"+ date
    try:
        urllib.request.urlretrieve(url, filename)
        print("found : " + date)
        unfilled = gzip.open(filename)
#         df_reviews[file_python] = pd.read_csv(unfilled)
        temp = pd.read_csv(unfilled, usecols=listing_col)
        temp['publicatie'] = date
        
        df_listings.append(temp)
        
        
    except urllib.error.URLError as e:
        print(e.reason) 
        

        


Not Found
Not Found
Not Found
Not Found
Not Found
Not Found
Not Found
Not Found
Not Found
Not Found
Not Found
Not Found
Not Found
Not Found
Not Found
Not Found
Not Found
Not Found
Not Found
Not Found
Not Found
Not Found
Not Found
found : 2018-02-03
Not Found
Not Found
Not Found
Not Found
Not Found
Not Found
Not Found
Not Found
Not Found
Not Found
Not Found
Not Found
Not Found
Not Found
Not Found
Not Found
Not Found
Not Found
Not Found
Not Found
Not Found
Not Found
Not Found
Not Found
Not Found
Not Found
Not Found
Not Found
Not Found
Not Found
Not Found
Not Found
Not Found
Not Found
found : 2018-03-10
Not Found
Not Found
Not Found
Not Found
Not Found
Not Found
Not Found
Not Found
Not Found
Not Found
Not Found
Not Found
Not Found
Not Found
Not Found
Not Found
Not Found
Not Found
Not Found
Not Found
Not Found
Not Found
Not Found
Not Found
Not Found
Not Found
Not Found
found : 2018-04-07
Not Found
Not Found
Not Found
Not Found
Not Found
Not Found
Not Found
Not Found
Not Found
Not Found
Not

## Making the dataframe

In [8]:
df_listings['price'] = pd.to_numeric(df_listings['price'].str.replace(',','').str.replace('$',''))

# Group by columns
price = df_listings.groupby(df_listings.neighbourhood_cleansed).price.mean()
listings =df_listings.groupby(df_listings.neighbourhood_cleansed).id.nunique()
accounts = df_listings.groupby(df_listings.neighbourhood_cleansed).host_id.nunique()

area_overview = pd.concat((price,listings,accounts), axis=1, join='inner').reset_index()
area_overview.rename(columns={'neighbourhood_cleansed': 'name', 'price': 'avg_price_night', 'id':'listings', 'host_id':'accounts'}, inplace=True)

## Tijdelijk dummy values voor volgende kolommen
area_overview["listings_per_ha"]  = area_overview["listings"] /3
area_overview["percentage_high_availability"] = area_overview["listings"] / 4
area_overview["percentage_host_with_multiple_listings"] = area_overview["listings"] /5
area_overview["duplicates"] = area_overview["listings"] / 6
area_overview["risk_rank"] = area_overview["listings"] / 7

## reorder the names
area_overview = area_overview.reindex(columns=['name','accounts','listings','listings_per_ha','avg_price_night','percentage_high_availability','percentage_host_with_multiple_listings','duplicates','risk_rank'])

## Show result
area_overview

,name,accounts,listings,listings_per_ha,avg_price_night,percentage_high_availability,percentage_host_with_multiple_listings,duplicates,risk_rank
0,Bijlmer-Centrum,90,105,35.000000,86.323810,26.25,21.0,17.500000,15.000000
1,Bijlmer-Oost,78,95,31.666667,90.357895,23.75,19.0,15.833333,13.571429
2,Bos en Lommer,948,1057,352.333333,109.568590,264.25,211.4,176.166667,151.000000
3,Buitenveldert - Zuidas,214,234,78.000000,126.611111,58.50,46.8,39.000000,33.428571
4,Centrum-Oost,1407,1673,557.666667,175.258816,418.25,334.6,278.833333,239.000000
5,Centrum-West,1780,2194,731.333333,177.758432,548.50,438.8,365.666667,313.428571
6,De Aker - Nieuw Sloten,106,123,41.000000,119.715447,30.75,24.6,20.500000,17.571429
7,De Baarsjes - Oud-West,2968,3322,1107.333333,134.080674,830.50,664.4,553.666667,474.571429
8,De Pijp - Rivierenbuurt,2157,2388,796.000000,141.879397,597.00,477.6,398.000000,341.142857
9,Gaasperdam - Driemond,91,113,37.666667,81.168142,28.25,22.6,18.833333,16.142857


In [21]:

# area_overview.to_json('area_overview.json' ,orient='records')


'{\'neighbourhoods\':[[{"name":"Bijlmer-Centrum","accounts":90,"listings":105,"listings_per_ha":35.0,"avg_price_night":86.3238095238,"percentage_high_availability":26.25,"percentage_host_with_multiple_listings":21.0,"duplicates":17.5,"risk_rank":15.0},{"name":"Bijlmer-Oost","accounts":78,"listings":95,"listings_per_ha":31.6666666667,"avg_price_night":90.3578947368,"percentage_high_availability":23.75,"percentage_host_with_multiple_listings":19.0,"duplicates":15.8333333333,"risk_rank":13.5714285714},{"name":"Bos en Lommer","accounts":948,"listings":1057,"listings_per_ha":352.3333333333,"avg_price_night":109.56859035,"percentage_high_availability":264.25,"percentage_host_with_multiple_listings":211.4,"duplicates":176.1666666667,"risk_rank":151.0},{"name":"Buitenveldert - Zuidas","accounts":214,"listings":234,"listings_per_ha":78.0,"avg_price_night":126.6111111111,"percentage_high_availability":58.5,"percentage_host_with_multiple_listings":46.8,"duplicates":39.0,"risk_rank":33.4285714286}